In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii


In [10]:
bounds = pickle.load(open('./pickle/bounds.pk','rb'))

In [11]:
master_cat = pd.read_csv('./catdata/master_catalog_jan_2023.csv')

In [12]:
cat_files = ['cat1_50.pk','cat51_100.pk','cat101_150.pk','cat151_200.pk','cat201_235.pk',
             'cat236_257.pk','cat258_279.pk','cat280_320.pk','cat321_360.pk','cat361_406.pk']

In [31]:
object_matches = pickle.load(open('./pickle/matches/matches_delta006.pk','rb'))

In [28]:
train = pickle.load(open('./pickle/training_data/training_data_d006_c300.pk','rb'))

In [15]:
def load_cat(field): # change to match-case
    if field <= 50: to_load = cat_files[0]
    elif field <= 100: to_load = cat_files[1]
    elif field <= 150: to_load = cat_files[2]
    elif field <= 200: to_load = cat_files[3]
    elif field <= 235: to_load = cat_files[4]
    elif field <= 257: to_load = cat_files[5]
    elif field <= 279: to_load = cat_files[6]
    elif field <= 320: to_load = cat_files[7]
    elif field <= 360: to_load = cat_files[8]
    elif field <= 406: to_load = cat_files[9]
    print(f'Loading {to_load} ...')
    catalogue = pickle.load(open(f'./pickle/{to_load}','rb'))
    
    return catalogue

# Test area

In [47]:
master_cat.loc[2619]

ID                             SDSS14
RADEG                        20.58625
DECDEG                      35.193056
CLASS                             4.0
DATATABLE    di_tullio_zinn_13_table1
Name: 2619, dtype: object

In [50]:
count = 0
for a in object_matches:
    for b in object_matches[a]:
        if b[2] < 10:
            count+=1
            print(b)
print(count)

(157325, 2619, 0)
(72446, 98, 0)
(66810, 1044, 0)
(174420, 2231, 0)
(114767, 139, 0)
(269225, 152, 0)
(69016, 155, 0)
(42680, 1814, 7)
(339006, 2330, 1)
(20233, 2075, 0)
(40570, 1986, 0)
(31943, 1990, 0)
(33485, 1999, 0)
(32606, 2007, 0)
(126676, 9, 0)
(1324801, 620, 0)
(345974, 119, 1)
(254699, 2373, 1)
(164295, 2374, 0)
(207126, 1989, 9)
(352428, 1999, 1)
(274981, 2007, 2)
(1457056, 146, 0)
(1326447, 257, 0)
(123066, 347, 0)
(86240, 2315, 0)
(379750, 2324, 9)
(379608, 2357, 0)
(511954, 1908, 0)
(185857, 2665, 0)
(55172, 2, 0)
(544195, 2083, 8)
(1734077, 749, 3)
(1711325, 750, 0)
(1343615, 756, 1)
(1337187, 777, 0)
(412557, 799, 0)
(1340891, 812, 9)
(1336235, 897, 1)
(391044, 901, 3)
(362768, 955, 0)
(1666613, 962, 7)
(1308249, 975, 0)
(348791, 978, 0)
(845195, 1000, 0)
(389923, 1040, 0)
(1274912, 1064, 0)
(823904, 1112, 9)
(1236496, 1209, 0)
(740954, 1426, 0)
(255920, 1467, 0)
(253268, 1586, 0)
(202894, 1664, 0)
(1251206, 477, 0)
(1179626, 684, 0)
(763446, 704, 0)
(724636, 756, 3)
(4

In [51]:

plt.hist

<function matplotlib.pyplot.hist(x, bins=None, range=None, density=False, weights=None, cumulative=False, bottom=None, histtype='bar', align='mid', orientation='vertical', rwidth=None, log=False, color=None, label=None, stacked=False, *, data=None, **kwargs)>

In [32]:
object_matches

{5: [(39800, 2647, 55)],
 11: [(118854, 2431, 33), (24692, 2319, 23)],
 13: [(43246, 2648, 51)],
 14: [(122860, 2644, 18), (73, 2648, 40)],
 20: [(157983, 2549, 20)],
 22: [(35300, 1454, 30)],
 24: [(190425, 2651, 34), (31036, 2652, 41), (99155, 2654, 25)],
 25: [(84044, 2656, 22), (40730, 2658, 22), (113222, 2654, 23)],
 26: [(220562, 2636, 46)],
 30: [(120326, 2636, 39)],
 33: [(155547, 2589, 33)],
 35: [(61764, 2570, 51)],
 37: [(29875, 1371, 25), (20014, 1766, 41)],
 43: [(123759, 2643, 150), (222746, 2652, 40)],
 52: [(37379, 2592, 48)],
 58: [(166727, 2642, 34), (159637, 2646, 34)],
 62: [(105959, 1214, 28)],
 70: [(107114, 2608, 85)],
 80: [(197399, 2527, 182)],
 81: [(140115, 102, 13)],
 82: [(4054, 2502, 22)],
 84: [(116154, 1335, 38)],
 85: [(178852, 2614, 31)],
 91: [(85976, 2597, 13)],
 92: [(44122, 18, 30)],
 94: [(159582, 2596, 32), (129761, 2591, 25)],
 95: [(31866, 8, 33), (31926, 7, 39)],
 97: [(135267, 2573, 24)],
 102: [(116568, 2538, 20)],
 111: [(56024, 2619, 36)],

# Generate Training Data

In [57]:
def generate_training_data(matches, crowding=300) -> pd.DataFrame:
    cat = load_cat(1)
    
    columns = ['obj_id','class','i','g','di','dg','ra','dec','field','pdidx']
    values = []
    object_ids = []
    
    #TEMP
    crowded_objects = []
    
    
    for field in matches: # iterate through each field ID
        print(f'Working on {field}')
        working_field = matches[field] # take the list of matches e.g. working_field = [(166727, 2642), (159637, 2646)]
        if field not in cat: # load the correct catalogue
            cat = load_cat(field)
        for m in working_field: # iterate through each match (a tuple) and grab values from catalogues
            
            if m[2] > crowding: # testing
                crowded_objects.append(m[1:])
                continue
            
            obj_id = master_cat.loc[m[1]].ID
            class_ = master_cat.loc[m[1]].CLASS
            
            if obj_id in object_ids: continue # if we've already added the object then skip
            else: object_ids.append(obj_id)   # else add it to the list of ids
            
            if class_ == 1: class_str = 'gc' # convert class numbers into strings
            elif class_ == 4: class_str = 'galaxy'
            elif class_ == 6: class_str = 'star'
            else: continue # skip non-gc/gal/stars
            
            # collect required data
            row = cat[field][m[0]]
            ra = row['RA']
            dec = row['Dec']
            g = row['g']
            i = row['i']
            dg = row['dg']
            di = row['di']
            
            values.append([obj_id,class_str,i,g,di,dg,ra,dec,field,m[0]])
    
    training_data_dict = dict(zip(columns,zip(*values))) # zip values and columns together into a dict (columns as keys)
    training_data_df = pd.DataFrame(training_data_dict) # put into pd Dataframe
    return training_data_df, crowded_objects

In [84]:
training_data, crowded_objects = generate_training_data(object_matches,crowding = 300)

Loading cat1_50.pk ...
Working on 5
Working on 11
Working on 13
Working on 14
Working on 20
Working on 22
Working on 24
Working on 25
Working on 26
Working on 30
Working on 33
Working on 35
Working on 37
Working on 43
Working on 52
Loading cat51_100.pk ...
Working on 58
Working on 62
Working on 70
Working on 80
Working on 81
Working on 82
Working on 84
Working on 85
Working on 91
Working on 92
Working on 94
Working on 95
Working on 97
Working on 102
Loading cat101_150.pk ...
Working on 111
Working on 114
Working on 124
Working on 125
Working on 126
Working on 128
Working on 138
Working on 139
Working on 142
Working on 146
Working on 148
Working on 153
Loading cat151_200.pk ...
Working on 162
Working on 165
Working on 167
Working on 169
Working on 173
Working on 180
Working on 183
Working on 184
Working on 185
Working on 186
Working on 187
Working on 188
Working on 189
Working on 196
Working on 201
Loading cat201_235.pk ...
Working on 204
Working on 205
Working on 207
Working on 208
Wor

In [85]:
with open('./pickle/training_data/training_data_d006_c300.pk2','wb') as f:
    pickle.dump(training_data,f)

# testing

In [86]:
training_data_c300_1 = pickle.load(open('./pickle/training_data/training_data_d006_c300.pk','rb'))

In [89]:
td1 = set(training_data_c300_1.obj_id)

In [88]:
td2 = set(training_data.obj_id)

### Zip testing

In [ ]:
#subset_matches = {k: object_matches[k] for k in (5,11,13,14)}

In [ ]:
set(training_data['class'])

In [18]:
a = [ [1,10], [2,20], [3,30] ]
c = ['dig','ten']

In [19]:
d = dict(zip(c,a))

In [20]:
dict(zip(c,zip(*a)))

{'dig': (1, 2, 3), 'ten': (10, 20, 30)}

In [25]:
list(zip(c,zip(*a)))

[('dig', (1, 2, 3)), ('ten', (10, 20, 30))]